In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn import metrics
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.ensemble import StackingRegressor
from scipy.stats import randint as sp_randint

import warnings
warnings.filterwarnings(action='ignore')



In [2]:
# get data

train = pd.read_csv('df - train_set.csv')
test = pd.read_csv('df - test_set.csv')

train = train[(train['Commodities'] == 'APPLE GOLDEN DELICIOUS')]
del train['Commodities'] 

train.head()

,Province,Container,Size_Grade,Weight_Kg,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg
1,CAPE,M4183,1L,18.3,2020-09-09,150.0,170.0,51710.0,332,6075.6,822,8.51
7,CAPE,JG110,2M,11.0,2020-04-14,50.0,50.0,16000.0,320,3520.0,0,4.55
24,W.CAPE-BERGRIVER ETC,JE090,2S,9.0,2020-04-16,55.0,55.0,990.0,18,162.0,1506,6.11
40,CAPE,M4183,1S,18.3,2020-05-04,80.0,120.0,32020.0,388,7100.4,443,4.51
69,EASTERN CAPE,IA400,1S,400.0,2020-09-28,1800.0,1800.0,1800.0,1,400.0,2,4.50


In [15]:
del test['Commodities']
test.head()

,Index,Province,Container,Size_Grade,Weight_Kg,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand
0,1,W.CAPE-BERGRIVER ETC,EC120,1M,12.0,2020-07-09,128.0,136.0,5008.0,38,456.0,0
1,2,W.CAPE-BERGRIVER ETC,M4183,1X,18.3,2020-01-20,220.0,220.0,1760.0,8,146.4,2
2,3,W.CAPE-BERGRIVER ETC,EC120,1S,12.0,2020-08-19,120.0,120.0,720.0,6,72.0,45
3,4,W.CAPE-BERGRIVER ETC,M4183,1M,18.3,2020-05-06,160.0,160.0,160.0,1,18.3,8
4,5,W.CAPE-BERGRIVER ETC,M4183,1L,18.3,2020-05-04,140.0,160.0,14140.0,100,1830.0,19


In [4]:
train.shape

(1952, 12)

In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685 entries, 0 to 684
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Index           685 non-null    int64  
 1   Province        685 non-null    object 
 2   Container       685 non-null    object 
 3   Size_Grade      685 non-null    object 
 4   Weight_Kg       685 non-null    float64
 5   Date            685 non-null    object 
 6   Low_Price       685 non-null    float64
 7   High_Price      685 non-null    float64
 8   Sales_Total     685 non-null    float64
 9   Total_Qty_Sold  685 non-null    int64  
 10  Total_Kg_Sold   685 non-null    float64
 11  Stock_On_Hand   685 non-null    int64  
dtypes: float64(5), int64(3), object(4)
memory usage: 64.3+ KB


In [6]:
train.describe()

,Weight_Kg,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg
count,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000,1952.000000
mean,40.460912,174.307377,215.648053,20053.533811,174.510758,2960.176332,408.393955,6.778893
std,99.655169,373.553578,433.546159,39005.069445,308.810797,6097.416527,724.450582,2.248744
min,3.000000,2.000000,5.000000,5.000000,1.000000,3.000000,0.000000,0.250000
25%,9.000000,50.000000,60.000000,1325.000000,12.000000,219.600000,9.000000,5.460000
50%,12.000000,80.000000,108.000000,5495.000000,64.000000,853.500000,126.500000,6.670000
75%,18.300000,127.250000,160.000000,21082.500000,200.000000,3093.525000,468.000000,8.280000
max,400.000000,2300.000000,3300.000000,369464.000000,4237.000000,74000.000000,6400.000000,21.240000


In [7]:
train['Province'].unique()

array(['CAPE', 'W.CAPE-BERGRIVER ETC', 'EASTERN CAPE', 'NATAL',
       'WEST COAST', 'TRANSVAAL', 'ORANGE FREE STATE'], dtype=object)

In [8]:
train['Container'].unique()

array(['M4183', 'JG110', 'JE090', 'IA400', 'EC120', 'AC030', 'M6125',
       'EF120', 'DT063', 'M9125', 'EG140'], dtype=object)

In [9]:
train['Size_Grade'].unique()

array(['1L', '2M', '2S', '1S', '1M', '1X', '2L', '2U', '2X', '1U'],
      dtype=object)

In [16]:
# DUMMY

def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column, drop_first=True)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [653]:
# PREPROCESSING 
seed = 1
def preprocess_inputs(df, return_df=False):
    df = df.copy()
    
    # CLEAN PROVINCE COLUMN
    
    df['Province'] = df['Province'].str.replace(' ', '_')
    df['Province'] = df['Province'].str.replace('.', '_')
    df['Province'] = df['Province'].str.replace('-', '_')
    
    # DATE ENCODING
    # Split 'Date' column into year, month and day columns 

    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.strftime('%d.%m.%Y')
    df['year'] = pd.DatetimeIndex(df['Date']).year
    df['month'] = pd.DatetimeIndex(df['Date']).month
    df['day'] = pd.DatetimeIndex(df['Date']).day

    df = df.drop(['Date'], axis = 1) 
    
    # BINARY ENCODING
    
    df['year'] = df['year'].replace({2020: 1, 2019: 0})
        
    # ONE-HOT ENCODING
    for column in ['Province', 'Container']:
        df = onehot_encode(df, column)
        
    # ORDINAL ENCODING
    enc = OrdinalEncoder()
    df[['Size_Grade']] = enc.fit_transform(df[['Size_Grade']])
        
    if return_df==True:
        ## for training dataset
        # REORDER COLUMNS SO THAT OUR DEPENDENT VARIABLE IS THE LAST COLUMN OF THE DATAFRAME
        if 'avg_price_per_kg' in df.columns:
            column_titles = [col for col in df.columns if col!= 'avg_price_per_kg'] + ['avg_price_per_kg']
            df = df.reindex(columns = column_titles)
            
        return df
    
        ## for training dataset
        # REORDER COLUMNS SO THAT OUR DEPENDENT VARIABLE IS THE LAST COLUMN OF THE DATAFRAME
    elif 'avg_price_per_kg' in df.columns:
        column_titles = [col for col in df.columns if col!= 'avg_price_per_kg'] + ['avg_price_per_kg']
        df = df.reindex(columns = column_titles)
    
        # SPLIT DATA INTO PREDICTORS AND TARGET
    
        y = df['avg_price_per_kg']
        X = df.drop('avg_price_per_kg', axis=1)
        y = np.array(y)
        

        
        # TRAIN TEST SPLIT
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, shuffle=False, random_state=seed)
        
        return X_train, X_test, y_train, y_test
    
        
    else:
        return df
    

In [654]:
X_train, X_test, y_train, y_test = preprocess_inputs(train)

In [655]:
# TRAIN A FEW MODELS

models = {
    "                         Decision Tree": DecisionTreeRegressor(random_state= seed),
    "                         Random Forest": RandomForestRegressor(min_samples_leaf= 1, n_estimators = 500, random_state= seed, max_depth = 13),
    "                     Gradient Boosting": GradientBoostingRegressor(learning_rate=0.2, n_estimators=6000, random_state= seed, max_depth =2),
    "                               XGBoost": XGBRegressor(max_depth=2,min_child_weight=13,subsample=1,colsample_bytree=1,
            objective='reg:squarederror',n_estimators=6000, learning_rate=0.1, random_state= seed),
    "                     CatBoostRegressor": CatBoostRegressor(verbose=0, learning_rate=0.1, depth = 4, iterations= 7000),
    "                         LGBMRegressor": LGBMRegressor()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

                         Decision Tree trained.
                         Random Forest trained.
                     Gradient Boosting trained.
                               XGBoost trained.
                     CatBoostRegressor trained.
                         LGBMRegressor trained.


In [656]:
#EVALUATE MODEL ON R SQUARED - HIGHER IS BETTER

for name, model in models.items():
    print(name + " R^2 Score: {:.5f}".format(model.score(X_test, y_test)))
    

                         Decision Tree R^2 Score: 0.87043
                         Random Forest R^2 Score: 0.94947
                     Gradient Boosting R^2 Score: 0.95698
                               XGBoost R^2 Score: 0.96824
                     CatBoostRegressor R^2 Score: 0.95384
                         LGBMRegressor R^2 Score: 0.94463


In [657]:
#EVALUATE MODEL ON RMSE - LOWER IS BETTER

for name, model in models.items():
    y_pred = model.predict(X_test)
    print(name + " Test RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_test ,y_pred))))
    
    y_train_pred = model.predict(X_train)
    print(name + " Train RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_train_pred ,y_train))))
    
    errors = abs(y_pred - y_test)

    # Display the performance metrics
    print('Mean Absolute Error:', round(np.mean(errors), 2), 'Rand.')

    mape = np.mean(100 * (errors / y_test))
    accuracy = 100 - mape

    print('Accuracy:', round(accuracy, 2), '%.')

                         Decision Tree Test RMSE: 0.82309
                         Decision Tree Train RMSE: 0.00000
Mean Absolute Error: 0.35 Rand.
Accuracy: 93.85 %.
                         Random Forest Test RMSE: 0.51399
                         Random Forest Train RMSE: 0.25615
Mean Absolute Error: 0.29 Rand.
Accuracy: 94.14 %.
                     Gradient Boosting Test RMSE: 0.47429
                     Gradient Boosting Train RMSE: 0.02034
Mean Absolute Error: 0.29 Rand.
Accuracy: 95.12 %.
                               XGBoost Test RMSE: 0.40748
                               XGBoost Train RMSE: 0.10494
Mean Absolute Error: 0.26 Rand.
Accuracy: 95.24 %.
                     CatBoostRegressor Test RMSE: 0.49127
                     CatBoostRegressor Train RMSE: 0.01002
Mean Absolute Error: 0.29 Rand.
Accuracy: 93.86 %.
                         LGBMRegressor Test RMSE: 0.53804
                         LGBMRegressor Train RMSE: 0.31678
Mean Absolute Error: 0.33 Rand.
Accuracy: 9

In [575]:
# GENERATE CSV FOR SUBMISSION TO KAGGLE
# DON'T SUBMIT ALL, ONLY THOSE WHICH DID GOOD ON EVALUATION TO CONFIRM

df = preprocess_inputs(test)

Xs = list(df.columns)
Xs.remove('Index')

X_test = df[important_list]

for name, model in models.items():
    y_pred = model.predict(X_test)
    d = pd.DataFrame(y_pred, columns =['avg_price_per_kg'])
    dff = pd.concat([df['Index'], d], axis=1)
    dff = dff.set_index('Index')
    
    match= re.findall('[A-Z]', name)    
    matchno = re.findall('[0-9]+', name)   
    file_name = ''.join(match) + ''.join(matchno)
    if file_name == 'XGB':
        dff.to_csv(file_name + '.csv')

# FEATURE SELECTION

In [658]:
# FEAUTURE IMPORTANCES

df = train.drop('avg_price_per_kg', axis = 1)
features = preprocess_inputs(df)
feature_list = list(features.columns)


for name, model in models.items():
    if name == "                               XGBoost":
        # Get numerical feature importances
        importances = list(model.feature_importances_)

        # List of tuples with variable and importance
        feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

        # Sort the feature importances by most important first
        feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

        # Print out the feature and importances 
        [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];



Variable: Weight_Kg            Importance: 0.5899999737739563
Variable: Low_Price            Importance: 0.11999999731779099
Variable: High_Price           Importance: 0.07999999821186066
Variable: Province_NATAL       Importance: 0.07999999821186066
Variable: year                 Importance: 0.03999999910593033
Variable: Province_W_CAPE_BERGRIVER_ETC Importance: 0.019999999552965164
Variable: Size_Grade           Importance: 0.009999999776482582
Variable: Sales_Total          Importance: 0.009999999776482582
Variable: Total_Kg_Sold        Importance: 0.009999999776482582
Variable: Stock_On_Hand        Importance: 0.009999999776482582
Variable: Province_EASTERN_CAPE Importance: 0.009999999776482582
Variable: Container_M4183      Importance: 0.009999999776482582
Variable: Total_Qty_Sold       Importance: 0.0
Variable: month                Importance: 0.0
Variable: day                  Importance: 0.0
Variable: Province_ORANGE_FREE_STATE Importance: 0.0
Variable: Province_TRANSVAAL   Imp

In [659]:
#SELECTING IMPORTANT FEATURES

df = preprocess_inputs(train, return_df=True)


y = df['avg_price_per_kg']
X = df.drop('avg_price_per_kg', axis=1)

# GET IMPORTANT COLUMN NAMES

important = []
for i in [ 'Total_Kg_Sold', 'Container_IA400', 'Container_M4183', "Container_JE090", 'Container_JG110', 
          'Weight_Kg', 'Total_Qty_Sold', 'High_Price', 'Sales_Total', 'Stock_On_Hand']:
    A = [col for col in df.columns if i in col]
    important.append(A)
    
important_list = [item for sublist in important for item in sublist]

# IMPORTANT DATAFRAME

X_imp = X[important_list]


In [660]:
# TEST NEW MODEL WITH IMPORTANT FEATURES ONLY

X_imp_train, X_imp_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.35, shuffle=False, random_state=seed)

xgb = XGBRegressor(max_depth=2,min_child_weight=13,subsample=1,colsample_bytree=1,
            objective='reg:squarederror',n_estimators=6000, learning_rate=0.1, random_state= seed)

xgb.fit(X_imp_train, y_train)

print("Trained.")

Trained.


In [661]:
# CHECK PERFORMANCE METRICS

pred = xgb.predict(X_imp_test)

errors = abs(pred - y_test)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'rand.')

mape = np.mean(100 * (errors / y_test))
accuracy = 100 - mape

print('Accuracy:', round(accuracy, 2), '%.')
print("Test RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_test ,pred))))
y_train_pred = xgb.predict(X_imp_train)
print("Train RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_train_pred ,y_train))))

Mean Absolute Error: 0.24 rand.
Accuracy: 95.45 %.
Test RMSE: 0.37198
Train RMSE: 0.12580


Lets compare all the models with their previous score rather.

In [662]:
# TRAIN MODELS AGAIN BUT WITH THE SUBSET CREATED ABOVE

X_imp_train, X_imp_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.35, shuffle=False, random_state=seed)
# TRAIN A FEW MODELS

models = {
    "                         Decision Tree": DecisionTreeRegressor(random_state= seed),
    "                         Random Forest": RandomForestRegressor(min_samples_leaf= 1, n_estimators = 500, random_state= seed, max_depth = 13),
    "                     Gradient Boosting": GradientBoostingRegressor(learning_rate=0.2, n_estimators=6000, random_state= seed, max_depth =2),
    "                               XGBoost": XGBRegressor(max_depth=2,min_child_weight=13,subsample=1,colsample_bytree=1,
            objective='reg:squarederror',n_estimators=6000, learning_rate=0.1, random_state= seed),
    "                     CatBoostRegressor": CatBoostRegressor(verbose=0, learning_rate=0.1, depth = 4, iterations= 7000),
    "                         LGBMRegressor": LGBMRegressor()
}

for name, model in models.items():
    model.fit(X_imp_train, y_train)
    print(name + " trained.")

                         Decision Tree trained.
                         Random Forest trained.
                     Gradient Boosting trained.
                               XGBoost trained.
                     CatBoostRegressor trained.
                         LGBMRegressor trained.


In [665]:
#EVALUATE SQUARED - HIGHER IS BETTER

for name, model in models.items():
    print(name + " R^2 Score: {:.5f}".format(model.score(X_imp_test, y_test)))
 

                         Decision Tree R^2 Score: 0.85566
                         Random Forest R^2 Score: 0.90141
                     Gradient Boosting R^2 Score: 0.96984
                               XGBoost R^2 Score: 0.97354
                     CatBoostRegressor R^2 Score: 0.96361
                         LGBMRegressor R^2 Score: 0.92886


In [666]:
#EVALUATE MODEL ON RMSE

X_imp_train, X_imp_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.35, shuffle=False, random_state=seed)

for name, model in models.items():
    y_pred = model.predict(X_imp_test)
    print(name + " Test RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_test ,y_pred))))
    
    y_train_pred = model.predict(X_imp_train)
    print(name + " Train RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error( y_train,y_train_pred))))
    
    errors = abs(y_pred - y_test)

    # Display the performance metrics
    print('Mean Absolute Error:', round(np.mean(errors), 2), 'Rand.')

    mape = np.mean(100 * (errors / y_test))
    accuracy = 100 - mape

    print('Accuracy:', round(accuracy, 2), '%.')

                         Decision Tree Test RMSE: 0.86874
                         Decision Tree Train RMSE: 0.00000
Mean Absolute Error: 0.52 Rand.
Accuracy: 91.46 %.
                         Random Forest Test RMSE: 0.71798
                         Random Forest Train RMSE: 0.33643
Mean Absolute Error: 0.47 Rand.
Accuracy: 91.64 %.
                     Gradient Boosting Test RMSE: 0.39711
                     Gradient Boosting Train RMSE: 0.03027
Mean Absolute Error: 0.26 Rand.
Accuracy: 95.52 %.
                               XGBoost Test RMSE: 0.37198
                               XGBoost Train RMSE: 0.12580
Mean Absolute Error: 0.24 Rand.
Accuracy: 95.45 %.
                     CatBoostRegressor Test RMSE: 0.43619
                     CatBoostRegressor Train RMSE: 0.02196
Mean Absolute Error: 0.27 Rand.
Accuracy: 94.9 %.
                         LGBMRegressor Test RMSE: 0.60990
                         LGBMRegressor Train RMSE: 0.36697
Mean Absolute Error: 0.4 Rand.
Accuracy: 92.

It apears the majority of the best performing models from our initial training have improved. Lets improve them a bit more.

# Ensemble Stacking

In [641]:
# NOW ENSEMBLE STACKING

# CHOOSE BEST MODELS FROM EARLIER SCORES

gb = GradientBoostingRegressor(learning_rate=0.2, n_estimators=6000, random_state= seed, max_depth =2)
xgb = XGBRegressor(max_depth=2,min_child_weight=13,subsample=1,colsample_bytree=1,
            objective='reg:squarederror',n_estimators=6000, learning_rate=0.1, random_state= seed)
cb = CatBoostRegressor(verbose=0, learning_rate=0.1, depth = 4, iterations= 7000, random_state= seed)

meta_learner_reg =  XGBRegressor(max_depth=2,min_child_weight=13,subsample=1,colsample_bytree=1, 
                                 objective='reg:squarederror', n_estimators=6000, learning_rate=0.1, random_state= seed)


models_4stacking = [("gb", gb),("xgb", xgb), ('cb', cb)]

s_reg = StackingRegressor(estimators=models_4stacking, final_estimator= meta_learner_reg, passthrough = True, cv= 4)


In [642]:
# TRAIN ON DATAFRAME WITH 7 COLUMNS(MOST IMPORTANT)

X_imp_train, X_imp_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.35, shuffle=False, random_state=seed)

s_reg.fit(X_imp_train,y_train)
print("Stacked model fitted.")

Stacked model fitted.


In [587]:
X_imp_train, X_imp_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.35, shuffle=False, random_state=seed)

In [644]:
# METRICS OF STACKING REGRESSOR

#X_train, X_test, y_train, y_test = preprocess_inputs(train) 0.41877

y_pred = s_reg.predict(X_imp_test)
rsq = s_reg.score(X_imp_test, y_test)
print("R^2 Score: ", rsq)

print("Test RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error( y_test,y_pred))))
y_train_pred = s_reg.predict(X_imp_train)
print("Train RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_train_pred ,y_train))))
 
errors = abs(y_pred - y_test)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
mape = np.mean(100 * (errors / y_test))
accuracy = 100 - mape
print('Accuracy:', round(accuracy, 2), '%.')

R^2 Score:  0.9928457311608316
Test RMSE: 0.16630
Train RMSE: 0.26121
Mean Absolute Error: 0.13 degrees.
Accuracy: 97.2 %.


In [590]:
df = preprocess_inputs(test)
Xs = list(df.columns)
Xs.remove('Index')

X_test = df[Xs]

x_t = X_test

y_pred = s_reg.predict(x_t)
d = pd.DataFrame(y_pred, columns =['avg_price_per_kg'])
dff = pd.concat([df['Index'], d], axis=1)
dff = dff.set_index('Index')
dff.to_csv('Stackedall.csv')

In [23]:
important = []
for i in ['Low_Price', 'Total_Kg_Sold', 'Container_IA400', 'Weight_Kg', 'Total_Qty_Sold', 'High_Price', 'Sales_Total']:
    A = [col for col in df.columns if i in col]
    important.append(A)
    
important_list = [item for sublist in important for item in sublist]

# IMPORTANT DATAFRAME
X_imp = X[important_list]

In [651]:
# USING KFOLD

df = preprocess_inputs(train, return_df=True)

important = []
for i in [ 'Total_Kg_Sold', 'Container_IA400', 'Container_M4183', "Container_JE090", 'Container_JG110', 
          'Weight_Kg', 'Total_Qty_Sold', 'High_Price', 'Sales_Total', 'Stock_On_Hand']:
    A = [col for col in df.columns if i in col]
    important.append(A)
    
important_list = [item for sublist in important for item in sublist]

# IMPORTANT DATAFRAME
X_imp = X[important_list]

def split_data_kf(df,K):
    
    y = df['avg_price_per_kg']
    X = df[important_list]  # SUBSET FOR IMPORTANT ROWS
    
    kf = KFold(n_splits=K, shuffle = False)
    indices = []
    
    for train_index, test_index in kf.split(X):
        indices.append((train_index, test_index))
        
    return indices


def get_best_kfmodel(df,data_indices):
    y = df['avg_price_per_kg']
    X = df[important_list]
    
    RMSE = []
    trainRSME =[]
    
    for (train_indices,test_indices) in data_indices:
        X_train, y_train = X.iloc[train_indices,:],y.iloc[train_indices]
        X_test, y_test = X.iloc[test_indices,:], y.iloc[test_indices]
        
        model = StackingRegressor(estimators=models_4stacking, final_estimator= meta_learner_reg, passthrough = True, cv= 4)
        
        model.fit(X_train, y_train) 

        y_pred = model.predict(X_test)     
        
        rmse = np.sqrt(metrics.mean_squared_error(y_test ,y_pred))
        RMSE.append(rmse)
        
        y_train_pred = model.predict(X_train)
        
        train_rsme = np.sqrt(metrics.mean_squared_error( y_train,y_train_pred))
        trainRSME.append(train_rsme)
        
    best = RMSE.index(min(RMSE))
    best_indicies = data_indices[best]
    
    
    X_train, y_train = X.iloc[best_indicies[0],:],y.iloc[best_indicies[0]]
    X_test, y_test = X.iloc[best_indicies[1],:], y.iloc[best_indicies[1]]
    
    model = StackingRegressor(estimators=models_4stacking, final_estimator= meta_learner_reg, passthrough = True, cv= 4)
    
    model.fit(X_train, y_train)       
          
    return model
       

In [649]:
df = preprocess_inputs(train, return_df = True)
data_indices = split_data_kf(df,4)

get_best_kfmodel(df,data_indices)

([0.2194026582554057,
  0.30308400059963264,
  0.2913206263331593,
  0.25979543207663186],
 [0.5238265379372173,
  0.36072228144873997,
  0.33778788271146826,
  0.3803486840052161])

In [652]:

train_df = preprocess_inputs(train, return_df = True)
data_indices = split_data_kf(train_df,4)
model = get_best_kfmodel(train_df,data_indices)

df = preprocess_inputs(test)
Xs = list(df.columns)
Xs.remove('Index')

X_test = df[Xs]
X_Test_imp = X_test[important_list]

y_pred = model.predict(X_Test_imp)
d = pd.DataFrame(y_pred, columns =['avg_price_per_kg'])
dff = pd.concat([df['Index'], d], axis=1)
dff = dff.set_index('Index')
dff.to_csv('stackedkfold.csv')

## Hyperparameter Tuning
### XGBoostRegressor

In [185]:

import xgboost as xgb

X_train, X_test, y_train, y_test = preprocess_inputs(train)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:squarederror',
}

params['eval_metric'] = "rmse"
num_boost_round = 999

model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10)


print("Best RMSE: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))


[0]	Test-rmse:3.84734
[1]	Test-rmse:2.63951
[2]	Test-rmse:1.84792
[3]	Test-rmse:1.30097
[4]	Test-rmse:0.91859
[5]	Test-rmse:0.65578
[6]	Test-rmse:0.48766
[7]	Test-rmse:0.38256
[8]	Test-rmse:0.32202
[9]	Test-rmse:0.29785
[10]	Test-rmse:0.29598
[11]	Test-rmse:0.28160
[12]	Test-rmse:0.28086
[13]	Test-rmse:0.28506
[14]	Test-rmse:0.29378
[15]	Test-rmse:0.29053
[16]	Test-rmse:0.29431
[17]	Test-rmse:0.29333
[18]	Test-rmse:0.28909
[19]	Test-rmse:0.28797
[20]	Test-rmse:0.28790
[21]	Test-rmse:0.28190
[22]	Test-rmse:0.27807
[23]	Test-rmse:0.27382
[24]	Test-rmse:0.27366
[25]	Test-rmse:0.26955
[26]	Test-rmse:0.26044
[27]	Test-rmse:0.24641
[28]	Test-rmse:0.24598
[29]	Test-rmse:0.24564
[30]	Test-rmse:0.24544
[31]	Test-rmse:0.24503
[32]	Test-rmse:0.24681
[33]	Test-rmse:0.24739
[34]	Test-rmse:0.24711
[35]	Test-rmse:0.24743
[36]	Test-rmse:0.24964
[37]	Test-rmse:0.25284
[38]	Test-rmse:0.25372
[39]	Test-rmse:0.25356
[40]	Test-rmse:0.25241
Best RMSE: 0.25 with 32 rounds


In [186]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=1,
    nfold=4,
    metrics={'rmse'},
    early_stopping_rounds=10
)
cv_results

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,4.739938,0.021968,4.743877,0.069658
1,3.377401,0.018440,3.386927,0.067667
2,2.427364,0.016886,2.445935,0.066945
3,1.766519,0.018177,1.800972,0.068197
4,1.311959,0.019460,1.367871,0.073691
...,...,...,...,...
161,0.017392,0.000828,0.577459,0.098167
162,0.017130,0.000941,0.577447,0.098183
163,0.016986,0.000961,0.577454,0.098178
164,0.016729,0.000872,0.577384,0.098205


In [188]:
cv_results['test-rmse-mean'].min()

0.57736325

In [ ]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(5,15)
    for min_child_weight in range(3,7)
]

# Define initial best params and RMSE
min_rmse = float("Inf")
best_params = None

for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=1,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10)
    
    # Update best RMSE
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (max_depth,min_child_weight)
        
print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))

In [168]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [10/10. for i in range(10,11)]
    for colsample in [10/10. for i in range(10,11)]
]

min_rmse = float("Inf")
best_params = None


for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(subsample, colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=1,
        nfold=4,
        metrics={'rmse'},
        early_stopping_rounds=10)
    
    # Update best score
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (subsample,colsample)
print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))


CV with subsample=1.0, colsample=1.0
	RMSE 0.62435875 for 129 rounds
Best params: 1.0, 1.0, RMSE: 0.62435875


In [169]:
%time

min_rmse = float("Inf")
best_params = None

for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    
    %time cv_results = xgb.cv(params, dtrain,num_boost_round=num_boost_round,seed=1, nfold=4, metrics=['rmse'], early_stopping_rounds=10)
    
    # Update best score
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds\n".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = eta
print("Best params: {}, RMSE: {}".format(best_params, min_rmse))

Wall time: 0 ns
CV with eta=0.3
Wall time: 582 ms
	RMSE 0.62435875 for 129 rounds

CV with eta=0.2
Wall time: 1.5 s
	RMSE 0.6204102499999999 for 385 rounds

CV with eta=0.1
Wall time: 2.33 s
	RMSE 0.6167677500000001 for 600 rounds

CV with eta=0.05
Wall time: 4.22 s
	RMSE 0.6189087499999999 for 998 rounds

CV with eta=0.01
Wall time: 3.8 s
	RMSE 0.6918565 for 998 rounds

CV with eta=0.005
Wall time: 3.84 s
	RMSE 0.7676310000000001 for 998 rounds

Best params: 0.1, RMSE: 0.6167677500000001


In [170]:
params['eta'] = .1

params

{'max_depth': 3,
 'min_child_weight': 6,
 'eta': 0.1,
 'subsample': 1.0,
 'colsample_bytree': 1.0,
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse'}

In [171]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)
print("Best RMSE: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))

[0]	Test-rmse:6.00210
[1]	Test-rmse:5.42201
[2]	Test-rmse:4.90507
[3]	Test-rmse:4.44281
[4]	Test-rmse:4.02841
[5]	Test-rmse:3.65767
[6]	Test-rmse:3.32693
[7]	Test-rmse:3.03055
[8]	Test-rmse:2.76351
[9]	Test-rmse:2.53166
[10]	Test-rmse:2.32450
[11]	Test-rmse:2.13956
[12]	Test-rmse:1.97227
[13]	Test-rmse:1.82383
[14]	Test-rmse:1.69787
[15]	Test-rmse:1.58494
[16]	Test-rmse:1.48119
[17]	Test-rmse:1.39624
[18]	Test-rmse:1.31767
[19]	Test-rmse:1.24548
[20]	Test-rmse:1.18780
[21]	Test-rmse:1.13900
[22]	Test-rmse:1.09268
[23]	Test-rmse:1.05977
[24]	Test-rmse:1.02049
[25]	Test-rmse:0.98746
[26]	Test-rmse:0.95597
[27]	Test-rmse:0.93827
[28]	Test-rmse:0.91373
[29]	Test-rmse:0.89372
[30]	Test-rmse:0.87661
[31]	Test-rmse:0.85689
[32]	Test-rmse:0.84373
[33]	Test-rmse:0.82737
[34]	Test-rmse:0.81667
[35]	Test-rmse:0.80478
[36]	Test-rmse:0.79101
[37]	Test-rmse:0.77966
[38]	Test-rmse:0.76753
[39]	Test-rmse:0.76075
[40]	Test-rmse:0.75490
[41]	Test-rmse:0.75041
[42]	Test-rmse:0.74318
[43]	Test-rmse:0.7385

[346]	Test-rmse:0.52550
[347]	Test-rmse:0.52507
[348]	Test-rmse:0.52508
[349]	Test-rmse:0.52533
[350]	Test-rmse:0.52528
[351]	Test-rmse:0.52516
[352]	Test-rmse:0.52508
[353]	Test-rmse:0.52478
[354]	Test-rmse:0.52472
[355]	Test-rmse:0.52452
[356]	Test-rmse:0.52440
[357]	Test-rmse:0.52399
[358]	Test-rmse:0.52373
[359]	Test-rmse:0.52354
[360]	Test-rmse:0.52338
[361]	Test-rmse:0.52334
[362]	Test-rmse:0.52317
[363]	Test-rmse:0.52253
[364]	Test-rmse:0.52243
[365]	Test-rmse:0.52232
[366]	Test-rmse:0.52235
[367]	Test-rmse:0.52245
[368]	Test-rmse:0.52246
[369]	Test-rmse:0.52215
[370]	Test-rmse:0.52203
[371]	Test-rmse:0.52199
[372]	Test-rmse:0.52197
[373]	Test-rmse:0.52169
[374]	Test-rmse:0.52160
[375]	Test-rmse:0.52138
[376]	Test-rmse:0.52143
[377]	Test-rmse:0.52132
[378]	Test-rmse:0.52103
[379]	Test-rmse:0.52087
[380]	Test-rmse:0.52074
[381]	Test-rmse:0.52072
[382]	Test-rmse:0.52077
[383]	Test-rmse:0.52070
[384]	Test-rmse:0.52077
[385]	Test-rmse:0.52065
[386]	Test-rmse:0.52031
[387]	Test-rmse:

[688]	Test-rmse:0.49187
[689]	Test-rmse:0.49186
[690]	Test-rmse:0.49195
[691]	Test-rmse:0.49184
[692]	Test-rmse:0.49182
[693]	Test-rmse:0.49174
[694]	Test-rmse:0.49163
[695]	Test-rmse:0.49171
[696]	Test-rmse:0.49163
[697]	Test-rmse:0.49159
[698]	Test-rmse:0.49162
[699]	Test-rmse:0.49148
[700]	Test-rmse:0.49130
[701]	Test-rmse:0.49114
[702]	Test-rmse:0.49115
[703]	Test-rmse:0.49109
[704]	Test-rmse:0.49101
[705]	Test-rmse:0.49090
[706]	Test-rmse:0.49111
[707]	Test-rmse:0.49117
[708]	Test-rmse:0.49119
[709]	Test-rmse:0.49116
[710]	Test-rmse:0.49119
[711]	Test-rmse:0.49117
[712]	Test-rmse:0.49110
[713]	Test-rmse:0.49109
[714]	Test-rmse:0.49104
[715]	Test-rmse:0.49102
Best RMSE: 0.49 in 706 rounds


In [172]:
num_boost_round = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

[0]	Test-rmse:6.00210
[1]	Test-rmse:5.42201
[2]	Test-rmse:4.90507
[3]	Test-rmse:4.44281
[4]	Test-rmse:4.02841
[5]	Test-rmse:3.65767
[6]	Test-rmse:3.32693
[7]	Test-rmse:3.03055
[8]	Test-rmse:2.76351
[9]	Test-rmse:2.53166
[10]	Test-rmse:2.32450
[11]	Test-rmse:2.13956
[12]	Test-rmse:1.97227
[13]	Test-rmse:1.82383
[14]	Test-rmse:1.69787
[15]	Test-rmse:1.58494
[16]	Test-rmse:1.48119
[17]	Test-rmse:1.39624
[18]	Test-rmse:1.31767
[19]	Test-rmse:1.24548
[20]	Test-rmse:1.18780
[21]	Test-rmse:1.13900
[22]	Test-rmse:1.09268
[23]	Test-rmse:1.05977
[24]	Test-rmse:1.02049
[25]	Test-rmse:0.98746
[26]	Test-rmse:0.95597
[27]	Test-rmse:0.93827
[28]	Test-rmse:0.91373
[29]	Test-rmse:0.89372
[30]	Test-rmse:0.87661
[31]	Test-rmse:0.85689
[32]	Test-rmse:0.84373
[33]	Test-rmse:0.82737
[34]	Test-rmse:0.81667
[35]	Test-rmse:0.80478
[36]	Test-rmse:0.79101
[37]	Test-rmse:0.77966
[38]	Test-rmse:0.76753
[39]	Test-rmse:0.76075
[40]	Test-rmse:0.75490
[41]	Test-rmse:0.75041
[42]	Test-rmse:0.74318
[43]	Test-rmse:0.7385

[346]	Test-rmse:0.52550
[347]	Test-rmse:0.52507
[348]	Test-rmse:0.52508
[349]	Test-rmse:0.52533
[350]	Test-rmse:0.52528
[351]	Test-rmse:0.52516
[352]	Test-rmse:0.52508
[353]	Test-rmse:0.52478
[354]	Test-rmse:0.52472
[355]	Test-rmse:0.52452
[356]	Test-rmse:0.52440
[357]	Test-rmse:0.52399
[358]	Test-rmse:0.52373
[359]	Test-rmse:0.52354
[360]	Test-rmse:0.52338
[361]	Test-rmse:0.52334
[362]	Test-rmse:0.52317
[363]	Test-rmse:0.52253
[364]	Test-rmse:0.52243
[365]	Test-rmse:0.52232
[366]	Test-rmse:0.52235
[367]	Test-rmse:0.52245
[368]	Test-rmse:0.52246
[369]	Test-rmse:0.52215
[370]	Test-rmse:0.52203
[371]	Test-rmse:0.52199
[372]	Test-rmse:0.52197
[373]	Test-rmse:0.52169
[374]	Test-rmse:0.52160
[375]	Test-rmse:0.52138
[376]	Test-rmse:0.52143
[377]	Test-rmse:0.52132
[378]	Test-rmse:0.52103
[379]	Test-rmse:0.52087
[380]	Test-rmse:0.52074
[381]	Test-rmse:0.52072
[382]	Test-rmse:0.52077
[383]	Test-rmse:0.52070
[384]	Test-rmse:0.52077
[385]	Test-rmse:0.52065
[386]	Test-rmse:0.52031
[387]	Test-rmse:

[688]	Test-rmse:0.49187
[689]	Test-rmse:0.49186
[690]	Test-rmse:0.49195
[691]	Test-rmse:0.49184
[692]	Test-rmse:0.49182
[693]	Test-rmse:0.49174
[694]	Test-rmse:0.49163
[695]	Test-rmse:0.49171
[696]	Test-rmse:0.49163
[697]	Test-rmse:0.49159
[698]	Test-rmse:0.49162
[699]	Test-rmse:0.49148
[700]	Test-rmse:0.49130
[701]	Test-rmse:0.49114
[702]	Test-rmse:0.49115
[703]	Test-rmse:0.49109
[704]	Test-rmse:0.49101
[705]	Test-rmse:0.49090


In [173]:
y_pred = best_model.predict(dtest)
print("Test RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_test ,y_pred))))
    
y_train_pred = best_model.predict(dtrain)
print("Train RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_train_pred ,y_train))))

Test RMSE: 0.49090
Train RMSE: 0.16178


In [176]:
params
#num_boost_round

{'max_depth': 3,
 'min_child_weight': 6,
 'eta': 0.1,
 'subsample': 1.0,
 'colsample_bytree': 1.0,
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse'}

In [196]:
X_train, X_test, y_train, y_test = preprocess_inputs(train)

max_depth = 3
min_child_weight = 6
subsample = 1
colsample_bytree = 1
objective = 'reg:squarederror'
num_estimators = 706
learning_rate = 0.1

xgb = XGBRegressor(max_depth=3,
                min_child_weight=6,
                subsample=1,
                colsample_bytree=1,
                objective='reg:squarederror',
                n_estimators=706,
                learning_rate=0.1, random_state= seed)

xgb.fit(X_train, y_train)
print('trained.')

trained.


In [197]:
y_pred = xgb.predict(X_test)
print("Test RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_test ,y_pred))))
    
y_train_pred = xgb.predict(X_train)
print("Train RMSE: {:.5f}".format(np.sqrt(metrics.mean_squared_error(y_train_pred ,y_train))))

Test RMSE: 0.49090
Train RMSE: 0.16178


In [ ]:
(max_depth=3,min_child_weight=3,subsample=1,colsample_bytree=1,
            objective='reg:squarederror',n_estimators=442, learning_rate=0.1, random_state= seed)

In [66]:
X_train, X_test, y_train, y_test = preprocess_inputs(train)